# Classify Irises

In this laboration we will try to build a neural net to identify what type of Iris a flower is. We will also evaluate the Artificial Neural Net by comparing it to other methods.

The dataset is a classical dataset in machinelearning collected in 1936 by Edgar Anderson and first published by Ronald Fisher. By measuring the sepal width, sepal length, petal width and petal length Fisher tried to specify differences between the spicies.

This is a simple example and often used when teaching ML algorithms. Since it contains of a reasonable amount of parameters it is easy to understand the data. It is also very good in the sense that there are some overlaps that might be very hard to detect accuratly. It therefor illustrates the complexity of building an AI that that has an accuracy of 100%.


![alt text](https://cdn-images-1.medium.com/max/2560/1*7bnLKsChXq94QjtAiRn40w.png "Three types of Irises the we want to classify.")

As you see those flowers are quite similar. Compared to a botanist the NN might not be perfect. But compared to someone who tries to identify the flower based on a flower book it might be quite accurate. Let's see how to do it.

In [ ]:
# First we load all libraries needed for this notebook
 
import pandas as pd # Load the Pandas libraries with alias 'pd'
import tensorflow as tf # A library developed by google making GPU calculations simple
import seaborn as sns # A nice library for plotting, alternative to matplotlib
from matplotlib import pyplot as plt # Another plotting library

from sklearn.preprocessing import normalize #
from sklearn.model_selection import train_test_split
from sklearn import preprocessing



In [ ]:
# Read data from file iris.csv' from the data folder
# Open the csv to see the data
try:
    # I you run local or on binder:
    data = pd.read_csv("./data/iris.csv") 
except:
    # If you run on colab
    data = pd.read_csv("https://raw.githubusercontent.com/simjoh/AI-Lab-Neural-Net-and-Python/master/data/iris.csv") 
# Preview the first 10 lines of the loaded data 
data.head()

## Explore the data
Before starting to do any modifications of the data or attempts to build some kind of AI/ML model. We must understand the data. This explorative phase is very important in order to succesful build a model. By understanding the data we might be able to improve the dataset.

One method in the pandas.DataFrame class is the `describe` method which shows some statistical information about each column.

In [ ]:
data.describe()

In [ ]:
cols=data.columns
sns.set(font_scale=2)
sns.pairplot(vars=cols[:-1], data=data, hue='Species', height=5)
plt.show()

### Delete data
By reading the data we can se that the parameter "Id" has a very strong correlation with the type of Iris. This is a pure artificial correlation existing only because of the structure of the data.

Id should therefor be deleted. One way to do this is with the `pop` method included in the dataframe class.


In [ ]:
try:
    data.pop('Id')
except:
    print('Id already deleted')
data.head(n=10)

In [ ]:
sns.pairplot(vars=cols[1:-1], data=data, hue='Species', height=5)
plt.show()

# Correlation of data
One method to analyze how well one variable describes another variable is by correlations.

$\rho(x,y)=\frac{\sum\limits_{i=1}^n(x_i-\bar{x})(y_i-\bar{y})}{\sum\limits_{i=1}^n(x_i-\bar{x})^2\sum\limits_{i=1}^n(y_i-\bar{y})^2}$

The correlation is a method included in the dataframe `corr`. 

In [ ]:
# Write the syntax here:
corr = data.corr()
corr

In [ ]:
import numpy as np
# Set up the matplotlib figure
sns.set(font_scale=1)
f, ax = plt.subplots(figsize=(7, 5))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, cmap=cmap, vmax=1,vmin=-1, center=0,# mask=mask,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})
plt.show()

From the the figure and table aboove we can see that two of the variables are well correlated. That means it is quite likely that two of the well correlated variables could be deleted without impacting the accuracy of the model.


In [ ]:
# Shuffle the data each time 
data = data.sample(frac=1).reset_index(drop=True)

# Split the data into 
X = data.iloc[:,:-1].values
y = data.iloc[:,-1].values
X_normalized = normalize(X,axis=0)

total_length = len(data)
train_length = int(0.8*total_length)
test_length = int(0.2*total_length)

X_train = X_normalized[:train_length]
X_test = X_normalized[train_length:]
y_train = y[:train_length]
y_test = y[train_length:]

le = preprocessing.LabelEncoder()
le.fit(y_train)
train_label = le.transform(y_train)
y_train = tf.keras.utils.to_categorical(train_label,num_classes=3)

le2 = preprocessing.LabelEncoder()
le2.fit(y_test)
test_label = le2.transform(y_test)
y_test=tf.keras.utils.to_categorical(test_label,num_classes=3)



In [ ]:
model=tf.keras.Sequential()
model.add(tf.keras.layers.Dense(200,input_dim=4,activation='relu'))
model.add(tf.keras.layers.Dense(100,activation='relu'))
model.add(tf.keras.layers.Dense(3,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])
history = model.fit(X_train,y_train,validation_data=(X_test,y_test),batch_size=20,epochs=300,verbose=1)


#### Validate results
By looking output from the model fit. We can se that the accuracy of training data keeps raising throuout the process of finding constants for each neuron in the net. However looking att the accuracy of the validation set there is some kind of maximum after quite few iteration. 

We can for a reasonable high amount epochs by plotting the history of the accuracy and loss. What is a reasonable amount of epochs?

In [ ]:
# summarize history for accuracy
sns.set(font_scale=1)
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

## Better net?
In the example above there are quite a lot of neurons in each layer of the net. What happens if we follow the rule of thumb and use somewhere between the amount of outputs classes and number of input variables in each layer? 

Do we see any diffrence in the loss and what happens to the accuracy? 

In [ ]:
model2=tf.keras.Sequential()
model2.add(tf.keras.layers.Dense(4,input_dim=4,activation='relu'))
model2.add(tf.keras.layers.Dropout(0.2))
model2.add(tf.keras.layers.Dense(3,activation='softmax'))
model2.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])
history2 = model2.fit(X_train,y_train,validation_data=(X_test,y_test),batch_size=20,epochs=500,verbose=1)


In [ ]:
# summarize history for accuracy
plt.plot(history2.history['acc'])
plt.plot(history2.history['val_acc'])
plt.title('model2 accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history2.history['loss'])
plt.plot(history2.history['val_loss'])
plt.title('model2 loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# Normalize the data before splitting it train test and validation set.
data = data.sample(frac=1).reset_index(drop=True)
data.iloc[:,:-1] = normalize(data.iloc[:,:-1].values,axis=0)

train, validate, test = np.split(data.sample(frac=1), [int(.6*len(data)), int(.8*len(data))])

# Split the data into train test and validation sets.
train_x = train.iloc[:,:-1].values
train_y = train.iloc[:,-1].values

test_x = test.iloc[:,:-1].values
test_y = test.iloc[:,-1].values

validate_x = validate.iloc[:,:-1].values
validate_y = validate.iloc[:,-1].values

# Convert the label to a understandable format for tensorflow
le = preprocessing.LabelEncoder()
le.fit(train_y)

train_label = le.transform(train_y)
train_y = tf.keras.utils.to_categorical(train_label,num_classes=3)

test_label = le.transform(test_y)
test_y = tf.keras.utils.to_categorical(test_label,num_classes=3)

validate_label = le.transform(validate_y)
validate_y = tf.keras.utils.to_categorical(validate_label, num_classes=3)

# What about another activation function
Usually Rectified Linear Unit (ReLU) is a good choice as activation function. In the example below we use the sigmoid function instead. Look at at the curves of the history and the accuracy and see how they compare with the curves above whe ReLU is used.

In [ ]:
model_sigmoid =tf.keras.Sequential()
model_sigmoid.add(tf.keras.layers.Dense(200,input_dim=4,activation='sigmoid'))
model_sigmoid.add(tf.keras.layers.Dense(100,activation='sigmoid'))
model_sigmoid.add(tf.keras.layers.Dense(3,activation='softmax'))
model_sigmoid.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])
history_sigmaid = model_sigmoid.fit(X_train,y_train,validation_data=(X_test,y_test),batch_size=20,epochs=300,verbose=1)


In [ ]:
# summarize history for accuracy
plt.plot(history_sigmaid.history['acc'])
plt.plot(history_sigmaid.history['val_acc'])
plt.title('model2 accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history_sigmaid.history['loss'])
plt.plot(history_sigmaid.history['val_loss'])
plt.title('model2 loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
import numpy as np
predicted_test = np.argmax(model_sigmoid.predict(test_x),axis=1)

predicted_validation = np.argmax(model_sigmoid.predict(validate_x),axis=1)
predicted_train = np.argmax(model_sigmoid.predict(train_x),axis=1)

print('True test : ', np.argmax(test_y,axis=1))
print('Pred test : ', predicted_test)
print()
print('True val  : ', np.argmax(validate_y,axis=1))
print('Pred val  : ', predicted_validation)
print()
print('True train: ', np.argmax(train_y,axis=1)[:30])
print('Pred train: ', predicted_train[:30])


In [ ]:
test['Prediction']=predicted_test==np.argmax(test_y,axis=1)

In [ ]:
sns.set(font_scale=1)
f, ax = plt.subplots(figsize=(7, 7))
plt.title('Note that variables are normalized')
sns.scatterplot(test.SepalLengthCm, test.SepalWidthCm, hue=test.Species, style=test.Prediction, legend='full' )
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()

##  Own work
Investigate how well the model predicts the training data and validation data

1. Make a new column in dataframe `train` and `validate` with the predicted value.
2. Plot each spicie with a different color and mark them with different markers if prediction is correct or not.

# Other algorithms
Random forest is another method in ML. The procedure for doing RF or NN is pretty similar and the results in this case is rather comparable. But notice that the time needed to train RF is significant shorter.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
clf = RandomForestClassifier(n_estimators =100)
clf.fit(train_x, train_y)
print('RF pred test: ', np.argmax(clf.predict(test_x),axis=1))
print('NN pred test: ', predicted_test)
print('True    test: ', np.argmax(test_y,axis=1))
print()
print('RF pred valid: ', np.argmax(clf.predict(validate_x),axis=1))
print('NN pred valid: ',  predicted_validation)
print('True    valid: ', np.argmax(validate_y,axis=1))
